<a href="https://colab.research.google.com/github/jisoolee11/study/blob/main/Dacon/computer_vision(test2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> DACON 제 2회 컴퓨터 비전 학습 경진대회

In [ ]:
# 드라이브에 마운트
from google.colab import drive
drive.mount('/content/gdrive')

# zip파일 풀기
!unzip -qq /content/drive/MyDrive/Data/computer_vision2/dirty_mnist_2nd.zip

In [ ]:
# 로컬파일 불러오기
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
cd /content/train

/content/train


In [ ]:
!unzip -qq /content/drive/MyDrive/Data/computer_vision2/dirty_mnist_2nd.zip

In [ ]:
cd /content/test

/content/test


In [ ]:
!unzip -qq /content/drive/MyDrive/Data/computer_vision2/test_dirty_mnist_2nd.zip

In [ ]:
cd ../

/content


In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import KFold

import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchsummary import summary
from torchvision import transforms
from torchvision.models import resnet101, resnet34

In [ ]:
# Out-of-Fold 전략을 위한 함수 정의
def split_dataset(path:os.PathLike) -> None:
    df = pd.read_csv(path)
    kfold = KFold(n_splits=5)
    for fold, (train, valid) in enumerate(kfold.split(df, df.index)):
        df.loc[valid, 'kfold'] = int(fold)

    df.to_csv('/content/split_kfold.csv', index = False)

In [ ]:
# 커스텀 데이터셋 정의
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
# trainset = MnistDataset('/content', '/content/gdrive/MyDrive/Data/computer_vision2/dirty_mnist_2nd_answer.csv', transforms_train)
# testset = MnistDataset('/content', '/content/gdrive/MyDrive/Data/computer_vision2/sample_submission.csv', transforms_test)

# train_loader = DataLoader(trainset, batch_size=32, num_workers=2)
# test_loader = DataLoader(testset, batch_size=10, num_workers=1)

In [ ]:
# 모형 정의
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet34()
        self.classifier = \
            nn.Linear(1000, 26)

        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x: Tensor) -> Tensor:
        x = self.resnet(x)
        x = self.classifier(x)

        return x

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
# model = MnistModel().to(device)
# print(summary(model,(3, 256, 256)))

In [ ]:
# 학습

def train(fold: int, verbose: int=30) -> None:
    split_dataset('/content/drive/MyDrive/Data/computer_vision2/dirty_mnist_2nd_answer.csv')
    df = pd.read_csv('/content/split_kfold.csv')
    df_train = df[df['kfold'] != fold].reset_index(drop=True)
    df_valid = df[df['kfold'] == fold].reset_index(drop=True)

    df_train.drop(['kfold'], axis=1).to_csv(f'/content/train-kfold-{fold}.csv', index=False)
    df_valid.drop(['kfold'], axis=1).to_csv(f'/content/valid-kfold-{fold}.csv', index=False)

    trainset = MnistDataset('/content/train', f'/content/train-kfold-{fold}.csv', transforms_train)
    train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)

    validset = MnistDataset('/content/train', f'/content/train-kfold-{fold}.csv', transforms_test)
    valid_loader = DataLoader(validset, batch_size=16, shuffle=False, num_workers=0)
    
    num_epochs = 80
    device = 'cuda'
    scaler = GradScaler()

    model = MnistModel().to(device)
    # model = nn.DataParallel(model, device_ids=[0])

    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    criterion = nn.MultiLabelSoftMarginLoss()

    for epoch in range(num_epochs):
        model.train()
        for i, (images, targets) in enumerate(train_loader):
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i+1) % 100 == 0:
                outputs = outputs > 0.0
                acc = (outputs == targets).float().mean()
                print(f'{{Fold {fold} | Epoch {epoch} | L:{loss.item():.7f} |A:{acc.item():.7f}}}')

        model.eval()
        valid_acc = 0.0
        valid_loss = 0.0
        for i, (images, targets) in enumerate(valid_loader):
            images = images.to(device)
            targets = targets.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)

            valid_loss += loss.item()
            outputs = outputs > 0.0
            valid_acc += (outputs == targets).float().mean()

        print(f'Fold {fold} | Epoch {epoch} | L:{valid_loss/(i+1):.7f} |A:{valid_acc/(i+1):.7f}\n')

        if epoch > num_epochs-20 and epoch < num_epochs-1:
            torch.save(model.state_dict(), f'resnet101-f{fold}-{epoch}.pth')

In [ ]:
if __name__ == '__main__':
    train(0)
    train(1)
    train(2)
    train(3)
    train(4)

In [ ]:
# 테스트셋 제출
def load_model(fold: int, epoch: int, device: torch.device = 'cuda') -> nn.Module:
    model = MnistModel().to(device)
    state_dict = {}
    for k, v in torch.load(f'resnet-f{fold}-{epoch}.pth').items():
        state_dict[k[7:]] = var

    model.load_state_dict(state_dict)

    return model 

In [ ]:
def test(device: torch.device = 'cuda'):
    submit = pd.read_csv('/content/drive/MyDrive/Data/computer_vision2/sample_submission.csv')

    model1 = load_model(0, 50)
    model2 = load_model(1, 50)
    model3 = load_model(2, 50)
    model4 = load_model(3, 50)
    model5 = load_model(4, 50)

    # model1 = nn.DataParallel(model1, device_ids=[0, 1, 2, 3])
    # model2 = nn.DataParallel(model2, device_ids=[0, 1, 2, 3])
    # model3 = nn.DataParallel(model3, device_ids=[0, 1, 2, 3])
    # model4 = nn.DataParallel(model4, device_ids=[0, 1, 2, 3])
    # model5 = nn.DataParallel(model5, device_ids=[0, 1, 2, 3])

    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    model5.eval()
    
    batch_size = test_loader.batch_size
    batch_index = 0
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)

        outputs1 = model1(images)
        outputs2 = model2(images)
        outputs3 = model3(images)
        outputs4 = model4(images)
        outputs5 = model5(images)

        outputs = (outputs1+outputs2+outputs3+outputs4+outputs5) / 5

        outputs = outputs > 0.0
        batch_index = i * batch_size
        submit.iloc[batch_index:batch_index+batch_size, 1:] = \
            outputs.long().squeeze(0).detach().cpu().numpy()

    submit.to_csv('/content/gdrive/MyDrive/Data/computer_vision2/resnet101-e50ekfold.csv', index=False)

if __name__ == '__main__':
    test()

In [ ]:
submit = pd.read_csv('/content/gdrive/MyDrive/Data/computer_vision2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/gdrive/MyDrive/Data/computer_vision2/submit11.csv', index=False)


from google.colab import files

files.download('submit2.csv')